## ORIGINAL CODE

In [ ]:
import cv2
import numpy as np

img = cv2.imread("docs.jpg",1)

width = min(int(img.shape[0]/6),int(img.shape[1]/6))
hight = max(int(img.shape[0]/6),int(img.shape[1]/6))

resize = cv2.resize(img, (width,(hight)))
img_ctr = resize.copy()

screen_res = 1280,720
scale_width = screen_res[0]/min(img.shape[1],img.shape[0])
scale_hight = screen_res[1]/max(img.shape[1],img.shape[0])
scale = min(scale_width,scale_hight)

output_width = min(int(img.shape[1]*scale),int(img.shape[0]*scale))
output_hight = max(int(img.shape[1]*scale),int(img.shape[0]*scale))

#cv2.namedWindow('Output', cv2.WINDOW_NORMAL)
#cv2.resizeWindow('Output', output_width, output_hight)

def empty(a):
    pass

#cv2.namedWindow("screen")
#cv2.createTrackbar("v1","screen",1,200,empty)
#cv2.createTrackbar("v2","screen",1,200,empty)
kernal = np.ones((5,5))

def process(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(5,5),1)
    #v1 = cv2.getTrackbarPos("v1","screen")
    #v2 = cv2.getTrackbarPos("v2","screen")
    canny = cv2.Canny(blur,50,50)
    dilate = cv2.dilate(canny, kernal, iterations = 2)
    erode = cv2.erode(dilate, kernal, iterations = 1)

    return canny,erode,gray

def getcontour(img):
    large = np.array([])
    maxArea = 0
    contour,hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE )
    for ctr in contour:
        area = cv2.contourArea(ctr)
        if area > 14000:
            cv2.drawContours(img_ctr,ctr,-1,(55,255,55),3)
            peri = cv2.arcLength(ctr, True)
            approx = cv2.approxPolyDP(ctr, 0.02*peri,True)
            if len(approx) == 4 and area > maxArea:
                large = approx
                maxArea = area
    cv2.drawContours(img_ctr,large,-1,(55,0,255),15)
    return large

def reorder(points):
    points = points.reshape((4,2))
    new_points = np.zeros((4,1,2),np.int32)
    add = np.sum(points, axis = 1)
    #print('add = ',add)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]

    sub = np.diff(points, axis = 1)
    new_points[1] = points[np.argmin(sub)]
    new_points[2] = points[np.argmax(sub)]
    #print('rearranged points =',new_points)

    return new_points

def cvtBlack(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #v1 = cv2.getTrackbarPos("v1","screen")
    (thres , black) = cv2.threshold(gray, 90, 255, cv2.THRESH_BINARY)
    return black
    
def crop(img,large):
    #print(large.shape)
    large = reorder(large)
    pt1 = np.float32(large)
    pt2 = np.float32([[0,0],[width,0],[0,hight],[width,hight]])
    matrix = cv2.getPerspectiveTransform(pt1,pt2)
    output = cv2.warpPerspective(img, matrix, (width,hight))
    cropped = output[10:output.shape[0]-10, 10:output.shape[1]-10]

    black = cvtBlack(cropped)

    return black,cropped

while True:
    canny,erode,gray = process(resize)
    large = getcontour(erode)
    img_crop,black = crop(resize,large)
    #cv2.imshow("img", resize)
    #cv2.imshow("gray", gray)
    #cv2.imshow("canny", canny)
    #cv2.imshow("erode", erode)
    #cv2.imshow("img_ctr", img_ctr)
    cv2.imshow('Output', img_crop)
    cv2.imshow('cropped',black)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite(name +'converted.jpg', img_crop)
        print('saved!!!!')
        break
    
cv2.destroyAllWindows()


### UPGRADE & RESIZE

In [2]:
import cv2
import numpy as np

def process(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    erode = cv2.erode(gray, None, iterations=1)
    canny = cv2.Canny(erode, 100, 200)
    return canny,erode,gray

def getcontour(img):
    contours, _ = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    largest_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02*perimeter, True)
        if len(approx) == 4:
            largest_contour = approx
            break
    return largest_contour

def reorder(points):
    points = points.reshape((4,2))
    new_points = np.zeros((4,1,2),np.int32)
    add = np.sum(points, axis = 1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    diff = np.diff(points, axis = 1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]
    return new_points

def crop(img, large):
    width = 600
    height = 800
    if large is None:
        return None
    large = reorder(large)
    pt1 = np.float32(large)
    pt2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    matrix = cv2.getPerspectiveTransform(pt1, pt2)
    img_crop = cv2.warpPerspective(img, matrix, (width, height))
    return img_crop

# read the image
img = cv2.imread("docs.jpg", 1)

# resize the image to 60% of its original size
resized = cv2.resize(img, None, fx=0.4, fy=0.4)

canny,erode,gray = process(resized)
large = getcontour(erode)
img_crop = crop(resized, large)

cv2.imshow("img", resized)
cv2.imshow("gray", gray)
cv2.imshow("erode", erode)
cv2.imshow("canny", canny)

if img_crop is not None:
    cv2.imshow("crop", img_crop)
    
cv2.waitKey(0)
cv2.destroyAllWindows()


### COMBINE TO 1 WINDOW

In [2]:
import cv2
import numpy as np

# function to process image
def process(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 1)
    canny = cv2.Canny(blur, 100, 200)
    kernel = np.ones((5,5))
    erode = cv2.erode(canny, kernel, iterations=1)
    return canny,erode,gray

# function to get largest contour
def getcontour(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    max_area = 0
    largest_contour = None
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 5000:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.02*peri, True)
            if area > max_area and len(approx) == 4:
                largest_contour = approx
                max_area = area
    return largest_contour

# function to reorder contour points
def reorder(points):
    points = points.reshape((4,2))
    new_points = np.zeros((4,1,2),np.int32)
    add = np.sum(points, axis = 1)
    new_points[0] = points[np.argmin(add)]
    new_points[3] = points[np.argmax(add)]
    diff = np.diff(points, axis = 1)
    new_points[1] = points[np.argmin(diff)]
    new_points[2] = points[np.argmax(diff)]
    return new_points

# function to crop image
def crop(img,large):
    if large is None:
        return None
    large = reorder(large)
    pt1 = np.float32(large)
    scale_percent = 40 # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    pt2 = np.float32([[0,0],[width,0],[0,height],[width,height]])
    matrix = cv2.getPerspectiveTransform(pt1,pt2)
    img_crop = cv2.warpPerspective(img, matrix, (width,height))
    return img_crop

# load image and process
img = cv2.imread("docs.jpg",1)
resize = cv2.resize(img,(0,0),fx=0.4,fy=0.4)
canny,erode,gray = process(resize)
large = getcontour(erode)
img_crop = crop(resize, large)

# show all outputs in a single window
output = np.hstack((canny, erode, gray, cv2.cvtColor(resize, cv2.COLOR_BGR2GRAY)))
cv2.imshow("All Outputs", output)

# show cropped image
if img_crop is not None:
    cv2.imshow("Cropped Image", img_crop)

cv2.waitKey(0)
cv2.destroyAllWindows()